In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 71.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a411b8743f90957c7145f7ebc76670a28d2eadea729849b3de32254dfb14a513
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

spark_session = SparkSession.builder.appName("Lab2").getOrCreate()
sc = spark_session._sc
spark_session

In [5]:
!head /content/drive/MyDrive/data/programming-languages.csv

name,wikipedia_url
A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)
A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)
A-0 System,https://en.wikipedia.org/wiki/A-0_System
A+,https://en.wikipedia.org/wiki/A%2B_(programming_language)
A++,https://en.wikipedia.org/wiki/A%2B%2B
ABAP,https://en.wikipedia.org/wiki/ABAP
ABC,https://en.wikipedia.org/wiki/ABC_(programming_language)
ABC ALGOL,https://en.wikipedia.org/wiki/ABC_ALGOL
ABSET,https://en.wikipedia.org/wiki/ABSET


In [6]:
progLangSchema = StructType([
    StructField("Language", StringType(), False),
    StructField("Url_language", StringType(), False),
])
dfProgLangs = spark_session.read.csv("/content/drive/MyDrive/data/programming-languages.csv", schema=progLangSchema)
dfProgLangs.head()

Row(Language='name', Url_language='wikipedia_url')

In [7]:
firstRow = dfProgLangs.rdd.first()
progLangsList = dfProgLangs.rdd\
    .filter(lambda x: x != firstRow)\
    .map(lambda x: x[0])\
    .collect()
progLangsList[:]

['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS',
 'ACC',
 'Accent',
 'Ace DASL',
 'ACL2',
 'ACT-III',
 'Action!',
 'ActionScript',
 'Ada',
 'Adenine',
 'Agda',
 'Agilent VEE',
 'Agora',
 'AIMMS',
 'Alef',
 'ALF',
 'ALGOL 58',
 'ALGOL 60',
 'ALGOL 68',
 'ALGOL W',
 'Alice',
 'Alma-0',
 'AmbientTalk',
 'Amiga E',
 'AMOS',
 'AMPL',
 'Apex (Salesforce.com)',
 'APL',
 "App Inventor for Android's visual block language",
 'AppleScript',
 'Arc',
 'ARexx',
 'Argus',
 'AspectJ',
 'Assembly language',
 'ATS',
 'Ateji PX',
 'AutoHotkey',
 'Autocoder',
 'AutoIt',
 'AutoLISP / Visual LISP',
 'Averest',
 'AWK',
 'Axum',
 'B',
 'Babbage',
 'Bash',
 'BASIC',
 'bc',
 'BCPL',
 'BeanShell',
 'Batch (Windows/Dos)',
 'Bertrand',
 'BETA',
 'Bigwig',
 'Bistro',
 'BitC',
 'BLISS',
 'Blockly',
 'BlooP',
 'Blue',
 'Boo',
 'Boomerang',
 'Bourne shell (including',
 'bash and',
 'ksh )',
 'BREW',
 'BPEL',
 'C',
 'C--',
 'C++ – ISO/IEC 14882',
 'C# – ISO/IEC

In [9]:
!head -n 3 /content/drive/MyDrive/data/posts_sample.xml

﻿<?xml version="1.0" encoding="utf-8"?>
<posts>
  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="630" ViewCount="42817" Body="&lt;p&gt;I want to use a track-bar to change a form's opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;'decimal'&lt;/code&gt; to &lt;code&gt;'double'&lt;/code&gt;&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn't work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="3641067" LastEditorDisplayName="Rich B" LastEditDate="2019-07-19T01:39:54.173" La

In [13]:
dfPostsSample = spark_session.read.format("xml").options(rowTag="row").load('/content/drive/MyDrive/data/posts_sample.xml')
print(dfPostsSample)
print("\n")
print(dfPostsSample.first())
print("\n")
print(dfPostsSample.head(2))

DataFrame[_AcceptedAnswerId: bigint, _AnswerCount: bigint, _Body: string, _ClosedDate: timestamp, _CommentCount: bigint, _CommunityOwnedDate: timestamp, _CreationDate: timestamp, _FavoriteCount: bigint, _Id: bigint, _LastActivityDate: timestamp, _LastEditDate: timestamp, _LastEditorDisplayName: string, _LastEditorUserId: bigint, _OwnerDisplayName: string, _OwnerUserId: bigint, _ParentId: bigint, _PostTypeId: bigint, _Score: bigint, _Tags: string, _Title: string, _ViewCount: bigint]


Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't 

In [14]:
def DefineLanguage(row):
    languageTag = None
    for language in progLangsList:
        if '<' + language.upper() + '>' in row._Tags.upper():
            languageTag = language
            break
    if languageTag is not None: 
        return (row._Id, languageTag, row._CreationDate.year)

def IsDateInRange(row):
    leftBorder = datetime(year=2010, month=1, day=1)
    rightBorder = datetime(year=2019, month=12, day=31)
    return row._CreationDate > leftBorder and row._CreationDate < rightBorder

In [15]:
type(dfPostsSample.rdd)

pyspark.rdd.RDD

In [16]:
topLanguagesPerYear = dfPostsSample.rdd.filter(lambda row: row._Tags is not None and IsDateInRange(row))\
    .map(DefineLanguage)\
    .filter(lambda row: row is not None)\
    .keyBy(lambda row: (row[2], row[1]))\
    .aggregateByKey(
        0,
        lambda count, value: count + 1,
        lambda count1, count2: count1 + count2,
    )\
    .map(lambda row: (row[0][0], row[0][1], row[1]))\
    .sortBy(lambda row: row[0])\
    .toDF(('Year', 'Language', 'Count'))
topLanguagesPerYear.show()  

+----+------------+-----+
|Year|    Language|Count|
+----+------------+-----+
|2010|      Python|   25|
|2010|  JavaScript|   44|
|2010|           R|    3|
|2010|      Delphi|    7|
|2010|        Perl|    3|
|2010|  PowerShell|    1|
|2010|       Mouse|    1|
|2010|     Haskell|    2|
|2010|       XPath|    1|
|2010|         PHP|   42|
|2010| AppleScript|    3|
|2010| Objective-C|   23|
|2010|       OCaml|    1|
|2010|ActionScript|    1|
|2010|         ksh|    1|
|2010|        Java|   52|
|2010|        Ruby|   11|
|2010|           C|   20|
|2010|         Sed|    1|
|2010|      MATLAB|    1|
+----+------------+-----+
only showing top 20 rows



In [17]:
pathLanguagesPer10Years = "parquets/languages_per_10_years.parquet"

In [20]:
topLanguagesPerYear.write.mode("overwrite").parquet(pathLanguagesPer10Years)

In [19]:
folderSaveName = "reports_top_10_languages_per_year"
os.makedirs(folderSaveName, exist_ok=True)

parquetFile = spark_session.read.parquet(pathLanguagesPer10Years)
parquetFile.createOrReplaceTempView("parquetFile")

for year in range(2010, 2020):
    print(year)
    top_ten_language = spark_session.sql(
        "SELECT Language, Count "
        "FROM parquetFile "
        f"WHERE Year = {year} "
        "ORDER BY Count DESC LIMIT 10 "
    )
    top_ten_language.write.mode("overwrite").parquet("%s/top_10_langueages_year_%s.parquet" %(folderSaveName, year))
    top_ten_language.show()

2010
+-----------+-----+
|   Language|Count|
+-----------+-----+
|       Java|   52|
| JavaScript|   44|
|        PHP|   42|
|     Python|   25|
|Objective-C|   23|
|          C|   20|
|       Ruby|   11|
|     Delphi|    7|
|          R|    3|
|       Perl|    3|
+-----------+-----+

2011
+-----------+-----+
|   Language|Count|
+-----------+-----+
|        PHP|   97|
|       Java|   92|
| JavaScript|   82|
|     Python|   35|
|Objective-C|   33|
|          C|   24|
|       Ruby|   17|
|     Delphi|    8|
|       Perl|    8|
|       Bash|    7|
+-----------+-----+

2012
+-----------+-----+
|   Language|Count|
+-----------+-----+
|        PHP|  131|
| JavaScript|  128|
|       Java|  119|
|     Python|   65|
|Objective-C|   44|
|          C|   26|
|       Ruby|   25|
|       Bash|    9|
|          R|    8|
|      Scala|    6|
+-----------+-----+

2013
+-----------+-----+
|   Language|Count|
+-----------+-----+
| JavaScript|  196|
|       Java|  191|
|        PHP|  173|
|     Python|   8